In [ ]:
!pip install transformers

In [ ]:
import json
from sklearn.model_selection import train_test_split

with open('/content/drive/MyDrive/ResearchProject/083/reviews_083_training.json') as f: #path to training data
    data = json.load(f)

train, test = train_test_split(data,test_size=0.15) 

def build_text_files_no_class(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for review in data_json:
        aspects = ""
        for pro in review["pros"]:
          aspects += f'<aspect>{pro}'
        for con in review["cons"]:
          aspects += f'<aspect>{con}'
        text = f'<startoftext><reviewtext>{review["text"]}<aspects>{aspects}<endoftext>'
        data += text + "\n"
    f.write(data)

build_text_files_no_class(train,'train_dataset.txt')
build_text_files_no_class(test,'test_dataset.txt')

print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("GroNLP/gpt2-small-dutch")

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

In [ ]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

In [ ]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("GroNLP/gpt2-small-dutch")

training_args = TrainingArguments(
    output_dir="./gpt2-without-classification", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32 # batch size for training
    per_device_eval_batch_size=64, # batch size for evaluation
    eval_steps = 200, # Number of update steps between two evaluations.
    save_steps=2000000, # after # steps model is saved 
    warmup_steps=250, # number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
trainer.train()
trainer.save_model()

In [ ]:
from transformers import pipeline

pipe = pipeline('text-generation', model='./gpt2-without-classification', tokenizer="GroNLP/gpt2-small-dutch")

In [ ]:
pipe("<startoftext><reviewtext>Geweldige wasmachine, draait hard. Maar wel luidruchtig. Ik vind het lawaai erg vervelend. De was wordt wel goed schoon. zeer mooie kleur<aspects>")

In [ ]:
pipe("<startoftext><reviewtext>De stoel zit lekker, maar ziet er niet uit.<aspects>")

In [ ]:
pipe("<startoftext><reviewtext>Mooi design maar de software is slecht<aspects>")

In [ ]:
pipe("<startoftext><reviewtext>mooi ding hoor<aspects>")

In [ ]:
# to download the model

from google.colab import files

!zip -r /content/gpt2-without-classification.zip /content/gpt2-without-classification
files.download('/content/gpt2-without-classification.zip')

# Load existing model

In [ ]:
from transformers import AutoModelWithLMHead
model = AutoModelWithLMHead.from_pretrained("/content/model")

In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer

pipe = pipeline('text-generation', model="/content/drive/MyDrive/ResearchProject/080/GPT2_without_class/model", tokenizer="GroNLP/gpt2-small-dutch")
tokenizer = AutoTokenizer.from_pretrained("GroNLP/gpt2-small-dutch")

# Evaluate model

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import json

with open('/content/drive/MyDrive/ResearchProject/083/reviews_083_testing.json') as f:
   test_data = json.load(f)

In [ ]:
#specify output path:
outpath = '/content/drive/MyDrive/ResearchProject/083/GPT2_without_class/GPT2_without_class_083_results.json'

def check_count():
  if count % 100 == 0:
    print("processed "+str(count)+" reviews out of " + str(total_num_reviews))
    with open(outpath, 'w') as outfile: 
      json.dump(test_data, outfile)


count = 0
total_num_reviews = len(test_data)

for review in test_data:
  if ("generated_aspects" in review.keys()) and review["generated_aspects"]:
    count += 1
    check_count()
    continue
  else:
    review["generated_aspects"] = []
  
  max_length = len(tokenizer(review["text"])["input_ids"]) + 15 + 15
  generated = pipe("<startoftext><reviewtext>" + review["text"] + "<aspects>", max_length = max_length)
  generated = generated[0]["generated_text"]
  generated = generated.replace(f'<startoftext><reviewtext>{review["text"]}<aspects>',"")

  if (not generated.startswith("<endoftext>")) and ("<endoftext>" in generated):
    generated = generated.split("<endoftext>")[0]
  else:
    continue

  generated = generated.split("<aspect>")
  for substring in generated:
    if substring:
      review["generated_aspects"].append(substring)
  count += 1
  check_count()


with open(outpath, 'w') as outfile:
  json.dump(test_data, outfile)

files.download(outpath)